In [1]:
import sys
import os

# Get the absolute path to the project directory
project_dir = os.path.abspath("..")

# Append the project directory to sys.path
if project_dir not in sys.path:
    sys.path.append(project_dir)

In [2]:
import yfinance as yf
import pandas as pd
import polars as pl
from typing import Dict
from dataclasses import asdict, is_dataclass, dataclass
import scipy
import numpy as np
from scipy.stats import linregress
import requests

from src.common.AssetData import AssetData
from src.common.AssetDataService import AssetDataService
from src.common.AssetFileInOut import AssetFileInOut 
from src.databaseService.OutsourceLoader import OutsourceLoader
from src.common.AssetFileInOut import AssetFileInOut
from src.common.AssetDataPolars import AssetDataPolars
from src.databaseService.CleanData import CleanData
from src.databaseService.Parser import Parser_AV
from src.common.YamlTickerInOut import YamlTickerInOut
from alpha_vantage.fundamentaldata import FundamentalData
from alpha_vantage.econindicators import EconIndicators
from alpha_vantage.alphaintelligence import AlphaIntelligence

from alpha_vantage.timeseries import TimeSeries
from alpha_vantage.fundamentaldata import FundamentalData
from alpha_vantage.econindicators import EconIndicators
from alpha_vantage.alphaintelligence import AlphaIntelligence


In [3]:
import os
print(os.getcwd())

c:\Users\kimer\Desktop\RandomOdyssey\notebooks


In [4]:
#load api key
import yaml
import os

# Define paths
current_dir = os.getcwd()
desired_folder = "secrets"
absolute_path_to_folder = os.path.join(os.path.abspath(os.path.join(current_dir, "..")), "secrets")

# Path to the YAML file
yaml_file_path = os.path.join("../secrets", "alphaVantage.yaml")

# Read and load the YAML file
try:
    with open(yaml_file_path, 'r') as file:  # Open the YAML file for reading
        config = yaml.safe_load(file)  # Load the YAML content
        apiKey = config['alphaVantage_premium']['apiKey']  # Access the required key
except PermissionError:
    print("Permission denied. Please check file permissions.")
except FileNotFoundError:
    print("File not found. Please verify the path.")
except KeyError:
    print("KeyError: Check the structure of the YAML file.")
except yaml.YAMLError as e:
    print("YAML Error:", e)

In [ ]:
import csv
import requests
from collections import defaultdict

# replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
CSV_URL = 'https://www.alphavantage.co/query?function=LISTING_STATUS&apikey='+apiKey

with requests.Session() as s:
    download = s.get(CSV_URL)
    decoded_content = download.content.decode('utf-8')
    cr = csv.reader(decoded_content.splitlines(), delimiter=',')
    my_list = list(cr)
    for row in my_list:
        print(row)
        
exchange_stocks = defaultdict(list)
with requests.Session() as s:
    download = s.get(CSV_URL)
    decoded_content = download.content.decode('utf-8')
    cr = csv.reader(decoded_content.splitlines(), delimiter=',')
    next(cr)  # skip header row
    for row in cr:
        if row[3].strip().lower() == 'stock' and row[6].strip().lower() == 'active':
            try:
                if int(row[4][:4]) < 2008:
                    exchange_stocks[row[2].strip()].append(row[0].strip())
            except ValueError:
                continue

# Create a list of dicts formatted for YAML
result = [{'stockExchange': exchange, 'stocks': stocks} for exchange, stocks in exchange_stocks.items()]

YamlTickerInOut('.').saveToFile(result, "stocks.yaml")

In [6]:
from alpha_vantage.timeseries import TimeSeries
from alpha_vantage.fundamentaldata import FundamentalData
from alpha_vantage.econindicators import EconIndicators
from alpha_vantage.alphaintelligence import AlphaIntelligence

ts = TimeSeries(key=apiKey, output_format='pandas')
fd = FundamentalData(key=apiKey, output_format='pandas')
ei = EconIndicators(key=apiKey, output_format='pandas')
ai = AlphaIntelligence(key=apiKey, output_format='pandas')

ticker="MBG.DEX"
fullSharePrice, _ = ts.get_daily_adjusted(symbol=ticker, outputsize='full')

url = 'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY_ADJUSTED&symbol='+ticker+'&apikey='+apiKey
r = requests.get(url)
data = r.json()
print(data)

{'Meta Data': {'1. Information': 'Daily Time Series with Splits and Dividend Events', '2. Symbol': 'MBG.DEX', '3. Last Refreshed': '2025-04-17', '4. Output Size': 'Compact', '5. Time Zone': 'US/Eastern'}, 'Time Series (Daily)': {'2025-04-17': {'1. open': '50.21', '2. high': '50.77', '3. low': '49.81', '4. close': '50.23', '5. adjusted close': '50.23', '6. volume': '1993456', '7. dividend amount': '0.0000', '8. split coefficient': '1.0'}, '2025-04-16': {'1. open': '49.58', '2. high': '50.39', '3. low': '49.075', '4. close': '50.39', '5. adjusted close': '50.39', '6. volume': '2300494', '7. dividend amount': '0.0000', '8. split coefficient': '1.0'}, '2025-04-15': {'1. open': '50.5', '2. high': '51.16', '3. low': '49.8', '4. close': '50.14', '5. adjusted close': '50.14', '6. volume': '2536682', '7. dividend amount': '0.0000', '8. split coefficient': '1.0'}, '2025-04-14': {'1. open': '49.12', '2. high': '49.52', '3. low': '48.675', '4. close': '49.26', '5. adjusted close': '49.26', '6. vol

the following returns {}

In [56]:
import requests
import json
ticker="AAPL"
year = 2022
url1 = 'https://www.alphavantage.co/query?function=NEWS_SENTIMENT&tickers='+ticker+'&time_from='+str(year)+'0101T0800&time_to='+str(year)+'1231T2300&limit=1000&apikey='+apiKey
url2 = 'https://www.alphavantage.co/query?function=NEWS_SENTIMENT&tickers=COIN,CRYPTO:BTC,FOREX:USD&time_from=20220410T0100&limit=1000&apikey='+apiKey
r1 = requests.get(url1)
data1 = r1.json()
print(json.dumps(data1, indent=2))

{
  "items": "684",
  "sentiment_score_definition": "x <= -0.35: Bearish; -0.35 < x <= -0.15: Somewhat-Bearish; -0.15 < x < 0.15: Neutral; 0.15 <= x < 0.35: Somewhat_Bullish; x >= 0.35: Bullish",
  "relevance_score_definition": "0 < x <= 1, with a higher score indicating higher relevance.",
  "feed": [
    {
      "title": "Apple, AMC, Meta, And How Elon Musk Is Reacting To Tesla Stock Plunge: Bulls And Bears Of The Week - Tesla  ( NASDAQ:TSLA ) ",
      "url": "https://www.benzinga.com/news/large-cap/22/12/30242858/bulls-and-bears-of-the-week-apple-amc-meta-and-how-elon-musk-is-reacting-to-tesla-stock-plunge",
      "time_published": "20221231T190310",
      "authors": [
        "Michael Cohen"
      ],
      "summary": "Benzinga examined the prospects for many investors' favorite stocks over the last week - here's a look at some of our top stories. The final week of trading in 2022 ended with a whimper as all three major indexes finished lower.",
      "banner_image": "https://cdn.be

In [6]:
import requests
import json
ticker="AAC"
url1 = 'https://www.alphavantage.co/query?function=OVERVIEW&symbol='+ticker+'&apikey='+apiKey
url2 = 'https://www.alphavantage.co/query?function=INCOME_STATEMENT&symbol='+ticker+'&apikey='+apiKey

r1 = requests.get(url1)
r2 = requests.get(url2)
data1 = r1.json()
data2 = r2.json()
print(json.dumps(data1, indent=2))
print(json.dumps(data2, indent=2))

{}
{}


In [7]:
tickerHandle = "CSCO"

company_overview, _ = fd.get_company_overview(symbol=tickerHandle)

url = 'https://www.alphavantage.co/query?function=INCOME_STATEMENT&symbol='+tickerHandle+'&apikey='+apiKey
incStatementData = requests.get(url).json()
url = 'https://www.alphavantage.co/query?function=CASH_FLOW&symbol='+tickerHandle+'&apikey='+apiKey
cashFlowData = requests.get(url).json()
url = 'https://www.alphavantage.co/query?function=BALANCE_SHEET&symbol='+tickerHandle+'&apikey='+apiKey
balanceSheetData = requests.get(url).json()
url = 'https://www.alphavantage.co/query?function=EARNINGS&symbol='+tickerHandle+'&apikey='+apiKey
earningsData = requests.get(url).json()

if incStatementData=={} or cashFlowData == {} or balanceSheetData == {} or earningsData == {}:
    raise ImportError(f"Empty Financial Data")

parser = Parser_AV(
    incStatementData=incStatementData, 
    cashFlowData=cashFlowData, 
    balanceSheetData=balanceSheetData, 
    earningsData=earningsData)

financials_annually, financials_quarterly = parser.to_pandas()

financials_quarterly = CleanData.financial_fiscalDateIncongruence(financials_quarterly, daysDiscrep = 60)
financials_annually = CleanData.financial_fiscalDateIncongruence(financials_annually, daysDiscrep = 330)

#financials_annually = CleanData.financial_dropDuplicateYears(financials_annually)
#financials_annually = CleanData.financial_dropLastRow(financials_annually)
#financials_annually = CleanData.financial_lastRow_removeIfOutOfFiscal(financials_annually)

In [8]:
start_idx = financials_annually['totalRevenue'].first_valid_index()
end_idx = financials_annually['totalRevenue'].last_valid_index()    
numeric_cols = financials_annually.select_dtypes(include=[np.number]).columns
non_numeric_cols = financials_annually.select_dtypes(exclude=[np.number]).columns

In [9]:
if np.isnan(financials_annually.loc[len(financials_annually)-1, 'totalRevenue']):
    financials_annually = CleanData.financial_lastRow_fillWithCompanyOverview_AV(financials_annually, company_overview)

In [37]:
financials_annually.loc[start_idx:, 'totalRevenue'] = (
    financials_annually.loc[start_idx:, 'totalRevenue']
    .interpolate(method='cubic')
)
financials_annually.loc[start_idx:, non_numeric_cols] = (
    financials_annually.loc[start_idx:, non_numeric_cols]
    .ffill()
)

In [28]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

imp_mean = IterativeImputer()
imp_mean.fit(financials_annually.loc[start_idx:, numeric_cols].to_numpy())
imputed_fin = imp_mean.transform(financials_annually.loc[start_idx:, numeric_cols].to_numpy())

# Convert the imputed array back to a DataFrame with the same index and columns
imputed_fin_df = pd.DataFrame(imputed_fin, columns=numeric_cols)

financials_annually.loc[start_idx:, numeric_cols] = imputed_fin_df

c:\Users\KILightTouch\Desktop\RandomOdyssey\.venv\Lib\site-packages\sklearn\impute\_base.py:598: UserWarning: Skipping features without any observed values: [76 79 82 84]. At least one non-missing value is needed for imputation with strategy='mean'.
  warnings.warn(
c:\Users\KILightTouch\Desktop\RandomOdyssey\.venv\Lib\site-packages\sklearn\impute\_base.py:598: UserWarning: Skipping features without any observed values: [76 79 82 84]. At least one non-missing value is needed for imputation with strategy='mean'.
  warnings.warn(


ValueError: Shape of passed values is (14, 84), indices imply (14, 88)

In [11]:
ticker="NVDA"
fullSharePrice, meta_data = ts.get_daily(symbol=ticker, outputsize='full')
fullSharePriceAdj, meta_data_Adj = ts.get_daily_adjusted(symbol=ticker, outputsize='full')
fullSharePriceAdj.rename(columns={
            '1. open': 'Open',
            '2. high': 'High',
            '3. low': 'Low',
            '4. close': 'Close',
            '5. adjusted close': 'Adj Close',
            '6. volume': 'Volume',
            '7. dividend amount': 'Dividends',
            '8. split coefficient': 'Splits'
        }, inplace=True)
fullSharePriceAdj_flipped = fullSharePriceAdj.iloc[::-1]
fullSharePriceAdj_flipped_index = fullSharePriceAdj_flipped.index
print(fullSharePriceAdj_flipped.index.name)

shareprice = fullSharePriceAdj_flipped[['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']]
volume = shareprice['Volume']
adjClosePrice = shareprice['Adj Close']

#company_overview, company_overview_2 = fd.get_company_overview(symbol=ticker)

income_statement_an, income_statement_an_2 = fd.get_income_statement_annual(symbol=ticker)
balance_sheet_an, balance_sheet_an_2 = fd.get_balance_sheet_annual(symbol=ticker)
cashFlow_an, cashFlow_an_2 = fd.get_cash_flow_annual(symbol=ticker)
earnings_an, earnings_an_2 = fd.get_earnings_annual(symbol=ticker)

income_statement_quad, income_statement_quad_2 = fd.get_income_statement_quarterly(symbol=ticker)
balance_sheet_quad, balance_sheet_quad_2 = fd.get_balance_sheet_quarterly(symbol=ticker)
cashFlow_quad, cashFlow_quad_2 = fd.get_cash_flow_quarterly(symbol=ticker)
earnings_quad, earnings_quad_2 = fd.get_earnings_quarterly(symbol=ticker)

inflation, _ = ei.get_inflation()

date


In [11]:
financials_an = pd.merge(earnings_an, income_statement_an, on="fiscalDateEnding", how="outer")
financials_an = financials_an.merge(balance_sheet_an, on="fiscalDateEnding", how="outer")
financials_an = financials_an.merge(cashFlow_an, on="fiscalDateEnding", how="outer")

In [28]:
import requests

ticker = "AAC"

url = 'https://www.alphavantage.co/query?function=INCOME_STATEMENT&symbol='+ticker+'&apikey='+apiKey
incStatementData = requests.get(url).json()
url = 'https://www.alphavantage.co/query?function=CASH_FLOW&symbol='+ticker+'&apikey='+apiKey
cashFlowData = requests.get(url).json()
url = 'https://www.alphavantage.co/query?function=BALANCE_SHEET&symbol='+ticker+'&apikey='+apiKey
balanceSheetData = requests.get(url).json()
url = 'https://www.alphavantage.co/query?function=EARNINGS&symbol='+ticker+'&apikey='+apiKey
earningstData = requests.get(url).json()

print(earningstData)

{}


In [26]:
df_ann_incStat = pd.DataFrame(incStatementData['annualReports'])
df_quar_incStat = pd.DataFrame(incStatementData['quarterlyReports'])
df_ann_cashFlow = pd.DataFrame(cashFlowData['annualReports'])
df_quar_cashFlow = pd.DataFrame(cashFlowData['quarterlyReports'])
df_ann_balSheet = pd.DataFrame(balanceSheetData['annualReports'])
df_quar_balSheet = pd.DataFrame(balanceSheetData['quarterlyReports'])
df_ann_earnings = pd.DataFrame(earningstData['annualEarnings'])
df_quar_earnings = pd.DataFrame(earningstData['quarterlyEarnings'])

exclude_cols = ['fiscalDateEnding','reportedDate', 'reportedCurrency', 'reportTime']
df_ann_incStat = df_ann_incStat.apply(lambda x: pd.to_numeric(x, errors='coerce') if x.name not in exclude_cols else x)
df_quar_incStat = df_quar_incStat.apply(lambda x: pd.to_numeric(x, errors='coerce') if x.name not in exclude_cols else x)
df_ann_cashFlow = df_ann_cashFlow.apply(lambda x: pd.to_numeric(x, errors='coerce') if x.name not in exclude_cols else x)
df_quar_cashFlow = df_quar_cashFlow.apply(lambda x: pd.to_numeric(x, errors='coerce') if x.name not in exclude_cols else x)
df_ann_balSheet = df_ann_balSheet.apply(lambda x: pd.to_numeric(x, errors='coerce') if x.name not in exclude_cols else x)
df_quar_balSheet = df_quar_balSheet.apply(lambda x: pd.to_numeric(x, errors='coerce') if x.name not in exclude_cols else x)
df_ann_earnings = df_ann_earnings.apply(lambda x: pd.to_numeric(x, errors='coerce') if x.name not in exclude_cols else x)
df_quar_earnings = df_quar_earnings.apply(lambda x: pd.to_numeric(x, errors='coerce') if x.name not in exclude_cols else x)

df_ann_incStat.replace('None', pd.NA, inplace=True)
df_quar_incStat.replace('None', pd.NA, inplace=True)
df_ann_cashFlow.replace('None', pd.NA, inplace=True)
df_quar_cashFlow.replace('None', pd.NA, inplace=True)
df_ann_balSheet.replace('None', pd.NA, inplace=True)
df_quar_balSheet.replace('None', pd.NA, inplace=True)
df_ann_earnings.replace('None', pd.NA, inplace=True)
df_quar_earnings.replace('None', pd.NA, inplace=True)

df_ann_incStat['fiscalDateEnding'] = pd.to_datetime(df_ann_incStat['fiscalDateEnding'], utc=True)
df_ann_incStat = df_ann_incStat.sort_values(by='fiscalDateEnding')
df_quar_incStat['fiscalDateEnding'] = pd.to_datetime(df_quar_incStat['fiscalDateEnding'], utc=True)
df_quar_incStat = df_quar_incStat.sort_values(by='fiscalDateEnding')
df_ann_cashFlow['fiscalDateEnding'] = pd.to_datetime(df_ann_cashFlow['fiscalDateEnding'], utc=True)
df_ann_cashFlow = df_ann_cashFlow.sort_values(by='fiscalDateEnding')
df_quar_cashFlow['fiscalDateEnding'] = pd.to_datetime(df_quar_cashFlow['fiscalDateEnding'], utc=True)
df_quar_cashFlow = df_quar_cashFlow.sort_values(by='fiscalDateEnding')
df_ann_balSheet['fiscalDateEnding'] = pd.to_datetime(df_ann_balSheet['fiscalDateEnding'], utc=True)
df_ann_balSheet = df_ann_balSheet.sort_values(by='fiscalDateEnding')
df_quar_balSheet['fiscalDateEnding'] = pd.to_datetime(df_quar_balSheet['fiscalDateEnding'], utc=True)
df_quar_balSheet = df_quar_balSheet.sort_values(by='fiscalDateEnding')
df_ann_earnings['fiscalDateEnding'] = pd.to_datetime(df_ann_earnings['fiscalDateEnding'], utc=True)
df_ann_earnings = df_ann_earnings.sort_values(by='fiscalDateEnding')
df_quar_earnings['fiscalDateEnding'] = pd.to_datetime(df_quar_earnings['fiscalDateEnding'], utc=True)
df_quar_earnings = df_quar_earnings.sort_values(by='fiscalDateEnding')

df_quar_earnings['reportedDate'] = pd.to_datetime(df_quar_earnings['reportedDate'], utc=True)

In [6]:
sec = company_overview["Sector"].iloc[0]
company_overview.to_dict(orient='records')[0]

{'Symbol': 'MSFT',
 'AssetType': 'Common Stock',
 'Name': 'Microsoft Corporation',
 'Description': "Microsoft Corporation is an American multinational technology company which produces computer software, consumer electronics, personal computers, and related services. Its best known software products are the Microsoft Windows line of operating systems, the Microsoft Office suite, and the Internet Explorer and Edge web browsers. Its flagship hardware products are the Xbox video game consoles and the Microsoft Surface lineup of touchscreen personal computers. Microsoft ranked No. 21 in the 2020 Fortune 500 rankings of the largest United States corporations by total revenue; it was the world's largest software maker by revenue as of 2016. It is considered one of the Big Five companies in the U.S. information technology industry, along with Google, Apple, Amazon, and Facebook.",
 'CIK': '789019',
 'Exchange': 'NASDAQ',
 'Currency': 'USD',
 'Country': 'USA',
 'Sector': 'TECHNOLOGY',
 'Indust

In [7]:
company_overview["Symbol"].iloc[0] 

'MSFT'

In [8]:
import requests
import json
symbol = 'ABBN.SW'
url = f'https://www.alphavantage.co/query'

# API parameters
params = {
    'function': 'TIME_SERIES_DAILY_ADJUSTED',
    'symbol': symbol,
    'apikey': apiKey,
    'datatype': 'pandas'
}

# Fetch the data
response = requests.get(url, params=params)
data = response.json()
df = pd.DataFrame.from_dict(data, orient='index')

Sentiment Analysis is to be implemented in the future!!!

In [9]:
import requests
import json
url = 'https://www.alphavantage.co/query?function=NEWS_SENTIMENT&tickers=MBG.DEX&apikey='+apiKey
url = 'https://www.alphavantage.co/query?function=SYMBOL_SEARCH&keywords=ABBN&apikey='+apiKey
r = requests.get(url)
data = r.json()

print(json.dumps(data, indent=2))

{
  "bestMatches": [
    {
      "1. symbol": "ABBNY",
      "2. name": "ABB Ltd",
      "3. type": "Equity",
      "4. region": "United States",
      "5. marketOpen": "09:30",
      "6. marketClose": "16:00",
      "7. timezone": "UTC-04",
      "8. currency": "USD",
      "9. matchScore": "0.8889"
    }
  ]
}


In [10]:
combined_financials = pd.merge(earnings_quad, income_statement_quad, on="fiscalDateEnding", how="outer")
combined_financials.merge(balance_sheet_quad, on="fiscalDateEnding", how="outer")
combined_financials.merge(cashFlow_quad, on="fiscalDateEnding", how="outer")

combined_financials['fiscalDateEnding'] = pd.to_datetime(combined_financials['fiscalDateEnding'])
combined_financials = combined_financials.sort_values(by='fiscalDateEnding')
combined_financials['reportedDate'] = pd.to_datetime(combined_financials['reportedDate'])

financials_an = pd.merge(earnings_an, income_statement_an, on="fiscalDateEnding", how="outer")
financials_an.merge(balance_sheet_an, on="fiscalDateEnding", how="outer")
financials_an.merge(cashFlow_an, on="fiscalDateEnding", how="outer")
financials_an['fiscalDateEnding'] = pd.to_datetime(financials_an['fiscalDateEnding'])
financials_an = financials_an.sort_values(by='fiscalDateEnding')

type(combined_financials['fiscalDateEnding'].iloc[3])

pandas._libs.tslibs.timestamps.Timestamp

In [11]:
# replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
CSV_URL = 'https://www.alphavantage.co/query?function=EARNINGS_CALENDAR&symbol=MSFT&apikey='+apiKey

with requests.Session() as s:
    download = s.get(CSV_URL)
    decoded_content = download.content.decode('utf-8')
    cr = csv.reader(decoded_content.splitlines(), delimiter=',')
    my_list = list(cr)
    for row in my_list:
        print(row)

NameError: name 'csv' is not defined

In [21]:
import csv
import requests
from collections import defaultdict

# replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
CSV_URL = 'https://www.alphavantage.co/query?function=LISTING_STATUS&apikey='+apiKey

with requests.Session() as s:
    download = s.get(CSV_URL)
    decoded_content = download.content.decode('utf-8')
    cr = csv.reader(decoded_content.splitlines(), delimiter=',')
    my_list = list(cr)
    for row in my_list:
        print(row)
        
exchange_stocks = defaultdict(list)
with requests.Session() as s:
    download = s.get(CSV_URL)
    decoded_content = download.content.decode('utf-8')
    cr = csv.reader(decoded_content.splitlines(), delimiter=',')
    next(cr)  # skip header row
    for row in cr:
        if row[3].strip().lower() == 'stock' and row[6].strip().lower() == 'active':
            try:
                if int(row[4][:4]) < 2008:
                    exchange_stocks[row[2].strip()].append(row[0].strip())
            except ValueError:
                continue

# Create a list of dicts formatted for YAML
result = [{'stockExchange': exchange, 'stocks': stocks} for exchange, stocks in exchange_stocks.items()]

YamlTickerInOut('.').saveToFile(result, "stocks.yaml")

['symbol', 'name', 'exchange', 'assetType', 'ipoDate', 'delistingDate', 'status']
['A', 'Agilent Technologies Inc', 'NYSE', 'Stock', '1999-11-18', 'null', 'Active']
['AA', 'Alcoa Corp', 'NYSE', 'Stock', '2016-10-18', 'null', 'Active']
['AAA', 'ALTERNATIVE ACCESS FIRST PRIORITY CLO BOND ETF ', 'NYSE ARCA', 'ETF', '2020-09-09', 'null', 'Active']
['AAAU', 'Goldman Sachs Physical Gold ETF', 'BATS', 'ETF', '2018-08-15', 'null', 'Active']
['AACG', 'ATA Creativity Global', 'NASDAQ', 'Stock', '2008-01-29', 'null', 'Active']
['AACT', 'Ares Acquisition Corporation II - Class A', 'NYSE', 'Stock', '2023-06-12', 'null', 'Active']
['AACT-U', 'Ares Acquisition Corporation II - Units (1 Ord Class A & 1/2 War)', 'NYSE', 'Stock', '2023-04-21', 'null', 'Active']
['AACT-WS', 'Ares Acquisition Corporation II - Warrants (01/01/9999)', 'NYSE', 'Stock', '2023-06-12', 'null', 'Active']
['AADI', 'Aadi Bioscience Inc', 'NASDAQ', 'Stock', '2017-08-08', 'null', 'Active']
['AADR', 'ADVISORSHARES DORSEY WRIGHT ADR E